# Chainerを使う準備をします
https://github.com/kmaehashi/chainer-colab

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install 'cupy-cuda80==4.0.0rc1' 'chainer==4.0.0rc1'
!pip install -U numpy
!pip install chainercv==0.8.0
#!git clone https://github.com/chainer/chainercv.git

# Install a Drive FUSE wrapper.
https://github.com/astrada/google-drive-ocamlfuse

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
1. 表示されたURLリンクへ飛んで表示されたトークンをコピー
1. 表示されたテキスト入力欄へペースト
1. Enter

In [0]:
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
もう一度
1. 表示されたURLリンクへ飛んで表示されたトークンをコピー
1. 表示されたテキスト入力欄へペースト
1. Enter

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

# Create a file in Drive.
# !echo "This newly created file will appear in your Drive file list." > drive/created.txt

import os
os.chdir('drive')

# ChainerCVで丸バツ三角を学習させたモデルのデモをみる
最後の`demo('all.png')`は適当にGoogle driveに保存した画像パスをくわせて下さい

In [0]:
!git clone https://github.com/kfjt/chainer-bbox.git

In [0]:
os.chdir('chainer-bbox')
#!python train.py --gpu 0
!python demo.py all.png

In [0]:
import matplotlib.pyplot as plot
import chainer

from chainercv.links import SSD300
from chainercv.datasets import voc_bbox_label_names
from chainercv import utils
from chainercv.visualizations import vis_bbox

from via_utils import via_bbox_label_names

def demo(img):
 model = SSD300(n_fg_class=len(voc_bbox_label_names),pretrained_model='via_model')
 chainer.cuda.get_device_from_id(0).use()
 model.to_gpu()

 img = utils.read_image(img, color=True)
 bboxes, labels, scores = model.predict([img])
 bbox, label, score = bboxes[0], labels[0], scores[0]
 vis_bbox(img, bbox, label, score, label_names=via_bbox_label_names)
 plot.show()

demo('all.png')